In [1]:
from dask.distributed import Client, LocalCluster
import bokeh
import geopandas as gpd
import scipy
import libpysal
import momepy
import json
from dask import delayed
import glob

/tmp/ipykernel_16859/2444415863.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
daskCluster = LocalCluster(threads_per_worker=2,
                n_workers=8, memory_limit='100GB')

client = Client(daskCluster)

client

2023-04-12 18:08:57,137 - distributed.worker_memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 15.58 GiB
2023-04-12 18:08:57,139 - distributed.worker_memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 15.58 GiB
2023-04-12 18:08:57,141 - distributed.worker_memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 15.58 GiB
2023-04-12 18:08:57,143 - distributed.worker_memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 15.58 GiB
2023-04-12 18:08:57,145 - distributed.worker_memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 15.58 GiB
2023-04-12 18:08:57,147 - distributed.worker_memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 15.58 GiB
2023-04-12 18:08:57,149 - distributed.worker_memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 15.58 GiB
2023-04-12 18

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 124.66 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34873,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 124.66 GiB
Comm: tcp://127.0.0.1:36629,Total threads: 2
Dashboard: http://127.0.0.1:37265/status,Memory: 15.58 GiB
Nanny: tcp://127.0.0.1:35617,


In [3]:

@delayed
def queen(chunk_num):
    
    chunk = gpd.read_parquet(f"./out/singapore/expanded_cells_in_chunk_{chunk_num}.pq")

    w = libpysal.weights.Queen.from_dataframe(chunk, geom_col='geometry')
    w3 = momepy.sw_high(k=3, weights=w)

    scipy.sparse.save_npz(f"./out/singapore/cluster_{chunk_num}_weight.npz", w.sparse)
    scipy.sparse.save_npz(f"./out/singapore/cluster_{chunk_num}_weight_w3.npz", w3.sparse)

In [ ]:
file_pattern = "./out/singapore/expanded_cells_in_chunk_*.pq"
file_list = glob.glob(file_pattern)
num_files = len(file_list)

queen_out = [queen(n1) for n1 in range(num_files)]
dask.compute(queen_out)

In [ ]:
daskCluster.close()
client.shutdown()